# Lab 4-2: Load Data

### Imports

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

torch.manual_seed(1)

### Low-level Implementation

In [10]:
# data
xy = np.loadtxt('data-01-test-score.csv', delimiter=',', dtype=np.float32)
x_data, y_data = xy[:, 0:-1], xy[:, -1]

x_train = torch.FloatTensor(x_data)
y_train = torch.FloatTensor(y_data)

# init model
W = torch.zeros((3, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):
    # H(x)
    hypothesis = x_train.matmul(W) + b
    
    # cost
    cost = torch.mean((hypothesis - y_train) ** 2)
    
    # cost로 H(X) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # print log
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, cost.item()
    ))

Epoch    0/20 Cost: 26811.960938
Epoch    1/20 Cost: 10594.790039
Epoch    2/20 Cost: 4598.844727
Epoch    3/20 Cost: 2381.967041
Epoch    4/20 Cost: 1562.316162
Epoch    5/20 Cost: 1259.258545
Epoch    6/20 Cost: 1147.200073
Epoch    7/20 Cost: 1105.759155
Epoch    8/20 Cost: 1090.427490
Epoch    9/20 Cost: 1084.748901
Epoch   10/20 Cost: 1082.639771
Epoch   11/20 Cost: 1081.850220
Epoch   12/20 Cost: 1081.548340
Epoch   13/20 Cost: 1081.427002
Epoch   14/20 Cost: 1081.372437
Epoch   15/20 Cost: 1081.342651
Epoch   16/20 Cost: 1081.321655
Epoch   17/20 Cost: 1081.304199
Epoch   18/20 Cost: 1081.288086
Epoch   19/20 Cost: 1081.272339
Epoch   20/20 Cost: 1081.256592


### High-level Implementation with `nn.Module`

In [49]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self, x):
        return self.linear(x)
    

class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = x_data.tolist()
        self.y_data = y_data.reshape(-1,1).tolist()
        
    def __len__(self):
        return len(self.x_data)
    
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        
        return x, y
    
dataset = CustomDataset()

dataloader = DataLoader(
    dataset,
    batch_size=10,
    shuffle=True
)
model = MultivariateLinearRegressionModel()

nb_epochs = 1000
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples
        
        # H(x)
        prediction = model(x_train)
        
        # cost
        cost = F.mse_loss(prediction, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    if epoch % 100 == 0:
        # print log
        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, batch_idx+1, len(dataloader), cost.item()
        ))

Epoch    0/1000 Batch 3/3 Cost: 39911.562500
Epoch  100/1000 Batch 3/3 Cost: 49866.398438
Epoch  200/1000 Batch 3/3 Cost: 42489.445312
Epoch  300/1000 Batch 3/3 Cost: 41953.542969
Epoch  400/1000 Batch 3/3 Cost: 45450.093750
Epoch  500/1000 Batch 3/3 Cost: 36421.542969
Epoch  600/1000 Batch 3/3 Cost: 35891.515625
Epoch  700/1000 Batch 3/3 Cost: 49770.726562
Epoch  800/1000 Batch 3/3 Cost: 41097.976562
Epoch  900/1000 Batch 3/3 Cost: 41705.476562
Epoch 1000/1000 Batch 3/3 Cost: 47547.320312
